<a href="https://colab.research.google.com/github/neuropoly/single_voxel_mrs_b0_shimming/blob/main/SVS_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVS processing - interactive notebook

This notebook is designed to reproduce the processing pipeline and visualize the results for the paper titled "On the Impact of B0 Shimming Algorithms on Single Voxel Magnetic Resonance Spectroscopy," authored by Behrouz Vejdani Afkham and Eva Alonso-Ortiz.

The data for this paper is available online on OSF and will be downloaded in the notebook to reproduce the results.

**Note**: While installing new packages, the user may be prompted to restart the session to use them. Please restart the session before proceeding to run the next cells.

## Contents:

1. [Install dependencies](#1.-Install-dependencies)
2. [Import required libraries](#2.-Import-required-libraries)
3. [Download the data](#3.-Download-the-data)
4. [B0 Fieldmaps calculation](#4.-Compute-fieldmaps)
5. [B0 shimming](#5.-Perform-B0-shimming-using-the-computed-filedmaps)
6. [B0 Fieldmaps comparison](#6.-B0-Fieldmaps-comparison)
7. [MRS analysis](#7.-MRS-analysis)


## 1. Install dependencies

In [ ]:
%cd /content
!git clone --recurse-submodules https://git.fmrib.ox.ac.uk/fsl/fsl_mrs.git
%cd /content/fsl_mrs
!pip install .

In [ ]:
%cd /content
!git clone --single-branch --branch gradient_optimizer https://github.com/shimming-toolbox/shimming-toolbox.git
%cd /content/shimming-toolbox/
!pip install .

## 2. Import required libraries

In [1]:
%cd /content
import os
os.environ["LD_PRELOAD"] = "";
os.environ["APPTAINER_BINDPATH"] = "/content"
os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"
os.environ["MODULEPATH"] = "/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/molecular_biology:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/workflows:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/visualization:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/structural_imaging:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/statistics:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/spine:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/spectroscopy:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/shape_analysis:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/segmentation:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/rodent_imaging:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/quantitative_imaging:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/quality_control:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/programming:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/phase_processing:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/machine_learning:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/image_segmentation:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/image_registration:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/image_reconstruction:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/hippocampus:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/functional_imaging:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/electrophysiology:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/diffusion_imaging:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/data_organisation:/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/body"

!curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
!chmod +x googlecolab_setup.sh
!./googlecolab_setup.sh


In [ ]:
import lmod
await lmod.load('fsl/6.0.4')
await lmod.list()

In [ ]:
import zipfile
import json
import os
import pathlib
import shutil
import datetime
import numpy as np
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.gridspec as gridspec
from shimmingtoolbox.prepare_fieldmap import prepare_fieldmap
from shimmingtoolbox.load_nifti import read_nii
from shimmingtoolbox.cli import b0shim

try:
    import seaborn as sns

except ImportError:
    !pip install seaborn
    import seaborn as sns

!pip install statsmodels
!pip install osfclient
print('Necessary libraries are imported')


## 3. Download the data

In [ ]:
!osf -p s3gwv fetch /SVS_MRS_DATA.zip

In [5]:
# Unzip the data
zipped_data = "/content/" + "SVS_MRS_DATA.zip"
with zipfile.ZipFile(zipped_data,"r") as zipfile:
  zipfile.extractall('/content/')

## 4. Compute fieldmaps

In [ ]:
if os.path.basename(os.getcwd()) != 'SVS_MRS_DATA':
    os.chdir(os.getcwd()+'/SVS_MRS_DATA/')

current_path = os.getcwd()
print(current_path)
shim_methods=['siemens','pi' ,'quadprog' ,'lsq','grad']
subjects_list = [name for name in os.listdir() if os.path.isdir(name) and name.startswith('sub')]
subjects_list.sort(key=lambda x: int(x.split('-')[1]))
print(subjects_list)

# Iterate over each subject
for subject in subjects_list:
    derivative_path = current_path + f"/derivatives/{subject}/"
    fmap_path = f"{derivative_path}fmap/"
    test_path = f"{fmap_path}test/"
    if not os.path.exists(fmap_path):
        os.makedirs(fmap_path)
    if not os.path.exists(test_path):
        os.makedirs(test_path)

    mask_mrs_nii = nib.load(derivative_path + 'mask_mrs.nii.gz')
    mask_mrs = mask_mrs_nii.get_fdata()
    mask_mrs[mask_mrs!=1]= np.nan
    mask_anat_nii = nib.load (derivative_path + 'bet_anat_mask.nii.gz')

    # Iterate over each shim method
    for method in shim_methods:
        gre_path =  current_path + f"/{subject}/fmap/"

        if method=="siemens":

            mag = read_nii(gre_path + f"{subject}_acq-siemens-after-Shim_magnitude1.nii.gz")
            phase1 = read_nii(gre_path + f"{subject}_acq-siemens-after-Shim_phase1.nii.gz", auto_scale=True)
            phase2 = read_nii(gre_path + f"{subject}_acq-siemens-after-Shim_phase2.nii.gz", auto_scale=True)

            # Load JSON data from the magnitude nifti
            with open(gre_path + f"{subject}_acq-siemens-after-Shim_magnitude1.json", 'r') as mag_json:
                json_data = json.load(mag_json)
            phase_list_data = [phase1[2], phase2[2]]

            # scale the phase data to (-pi, pi) range
            scaled_phase_list_data = [((phase_list_data[i] - np.min(phase_list_data[i]))/(np.max(phase_list_data[i])-np.min(phase_list_data[i])))
                                 *2*np.pi-np.pi for i in range(len(phase_list_data))]
            # convert to nifti
            scaled_phase_list_nii = [nib.Nifti1Image(scaled_phase_list_data[i], phase1[0].affine, phase1[0].header)
                                 for i in range(len(scaled_phase_list_data))]

            EchoTime1 = phase1[1]['EchoTime']
            EchoTime2 = phase2[1]['EchoTime']
            echo_times = [EchoTime1, EchoTime2]

            # compute fieldmap for each method
            fieldmap = prepare_fieldmap(scaled_phase_list_nii, echo_times, mag[2], unwrapper='prelude', gaussian_filter= True, nii_mask= mask_anat_nii, sigma=0.5)

            # store the fieldmap under the corresponding path
            fieldmap_nii = nib.Nifti1Image(fieldmap[0]*mask_mrs, phase1[0].affine, phase1[0].header )
            nib.save(fieldmap_nii, fmap_path + f"{subject}_acq-siemens-after-Shim-fieldmap.nii.gz")

            # store the corresponding json file for each fieldmap
            with open(fmap_path + f"{subject}_acq-siemens-after-Shim-fieldmap.json", 'w') as fmap_json:
                json.dump(json_data, fmap_json)

        else:
            mag_before= read_nii(gre_path + f"{subject}_acq-{method}-before-Shim_magnitude1.nii.gz")
            phase1_before = read_nii(gre_path + f"{subject}_acq-{method}-before-Shim_phase1.nii.gz", auto_scale=True)
            phase2_before = read_nii(gre_path + f"{subject}_acq-{method}-before-Shim_phase2.nii.gz", auto_scale=True)

            mag_after = read_nii(gre_path + f"{subject}_acq-{method}-after-Shim_magnitude1.nii.gz")
            phase1_after = read_nii(gre_path + f"{subject}_acq-{method}-after-Shim_phase1.nii.gz", auto_scale=True)
            phase2_after = read_nii(gre_path + f"{subject}_acq-{method}-after-Shim_phase2.nii.gz", auto_scale=True)

            # Load JSON data from the magnitude nifti
            with open(gre_path + f"{subject}_acq-{method}-before-Shim_magnitude1.json", 'r') as mag_json_before:
                json_data_before = json.load(mag_json_before)
            # Load JSON data from the magnitude nifti
            with open(gre_path + f"{subject}_acq-{method}-after-Shim_magnitude1.json", 'r') as mag_json_after:
                json_data_after = json.load(mag_json_after)

            phase_list_data_before = [phase1_before[2], phase2_before[2]]
            phase_list_data_after = [phase1_after[2], phase2_after[2]]

            # scale the phase data to (-pi, pi) range
            scaled_phase_list_data_before = [((phase_list_data_before[i] - np.min(phase_list_data_before[i]))/(np.max(phase_list_data_before[i])-np.min(phase_list_data_before[i])))
                                 *2*np.pi-np.pi for i in range(len(phase_list_data_before))]

            scaled_phase_list_data_after = [((phase_list_data_after[i] - np.min(phase_list_data_after[i]))/(np.max(phase_list_data_after[i])-np.min(phase_list_data_after[i])))
                     *2*np.pi-np.pi for i in range(len(phase_list_data_after))]

            # convert to nifti
            scaled_phase_list_nii_before = [nib.Nifti1Image(scaled_phase_list_data_before[i], phase1_before[0].affine, phase1_before[0].header)
                                 for i in range(len(scaled_phase_list_data_before))]

            scaled_phase_list_nii_after = [nib.Nifti1Image(scaled_phase_list_data_after[i], phase1_after[0].affine, phase1_after[0].header)
                                 for i in range(len(scaled_phase_list_data_after))]

            EchoTime1 = phase1[1]['EchoTime']
            EchoTime2 = phase2[1]['EchoTime']
            echo_times = [EchoTime1, EchoTime2]

            # compute fieldmap for each method
            fieldmap_before = prepare_fieldmap(scaled_phase_list_nii_before, echo_times, mag_before[2], unwrapper='prelude', gaussian_filter= True, nii_mask= mask_anat_nii, sigma=0.5)
            fieldmap_after = prepare_fieldmap(scaled_phase_list_nii_after, echo_times, mag_after[2], unwrapper='prelude', gaussian_filter= True, nii_mask= mask_anat_nii, sigma=0.5)

            # store the fieldmap under the corresponding path
            fieldmap_nii_after = nib.Nifti1Image(fieldmap_after[0]*mask_mrs, phase1_after[0].affine, phase1_after[0].header )
            nib.save(fieldmap_nii_after, fmap_path + f"{subject}_acq-{method}-after-Shim-fieldmap.nii.gz")

            fieldmap_nii_before = nib.Nifti1Image(fieldmap_before[0], phase1_before[0].affine, phase1_before[0].header )
            nib.save(fieldmap_nii_before, fmap_path + f"{subject}_acq-{method}-before-Shim-fieldmap.nii.gz")

            # store the corresponding json file for each fieldmap
            with open(fmap_path + f"{subject}_acq-{method}-before-Shim-fieldmap.json", 'w') as fmap_json_before:
                json.dump(json_data_before, fmap_json_before)

            with open(fmap_path + f"{subject}_acq-{method}-after-Shim-fieldmap.json", 'w') as fmap_json_after:
                json.dump(json_data_after, fmap_json_after)


In [ ]:
### Display field maps for Siemens and Grad methods for a representative subject.

fig = plt.figure(figsize=(10, 3), facecolor='white')

mask_mrs_sub3 = nib.load(current_path + '/derivatives/sub-3/mask_mrs.nii.gz').get_fdata()
mask_mrs_sub3[mask_mrs_sub3 != 1] = np.nan
fieldmap_Grad_sub3 = nib.load(current_path+"/derivatives/sub-3/fmap/sub-3_acq-grad-after-Shim-fieldmap.nii.gz").get_fdata()*mask_mrs_sub3
fieldmap_Siemens_sub3 = nib.load(current_path+"/derivatives/sub-3/fmap/sub-3_acq-siemens-after-Shim-fieldmap.nii.gz").get_fdata()*mask_mrs_sub3
magnitude = nib.load(current_path+"/sub-3/fmap/sub-3_acq-grad-before-Shim_magnitude1.nii.gz").get_fdata()

plt.suptitle(' Comparison of the measured fieldmaps [Hz] after shimming', fontsize=14, fontweight='bold')
# Define gridspec
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])

# Plot a sagittal cut for Grad method
ax1 = plt.subplot(gs[0])
ax1.imshow(np.rot90(magnitude[31,:,:]), cmap='gray')
img1= ax1.imshow(np.rot90(fieldmap_Grad_sub3[31,:,:]), vmin=-20, vmax=20, cmap='jet')

ax1.set_xticks([])  # Turn off x-axis ticks
ax1.set_yticks([])  # Turn off y-axis ticks
ax1.set_title('Grad', fontsize=12)
cbar = plt.colorbar(img1, ax= ax1, fraction=0.025, pad=0.03)
cbar.ax.tick_params(labelsize=12)  # Set font size for color bar

# Plot a sagittal cut for the Siemens method
ax2 = plt.subplot(gs[1])
ax2.imshow(np.rot90(magnitude[31,:,:]), cmap='gray')
img2 = ax2.imshow(np.rot90(fieldmap_Siemens_sub3[31,:,:]), vmin=-20, vmax=20, cmap='jet')

ax2.set_xticks([])  # Turn off x-axis ticks
ax2.set_yticks([])  # Turn off y-axis ticks
ax2.set_title('Siemens', fontsize=12)

# cbar = plt.colorbar(img, ax=ax2)  # Add color bar
cbar = plt.colorbar(img2, ax=ax2, fraction=0.025, pad=0.03)
cbar.ax.tick_params(labelsize=12)  # Set font size for color bar

# Save the figure with 300 dpi
plt.savefig('fieldmaps_after_shimming.png', dpi=300)
plt.show()
print(magnitude.shape)

## 5. Perform B0 shimming using the computed filedmaps and MRS voxel mask

In [ ]:
%%bash
# Generate subjects list as an array
subjects_list=$(find .  -type d -maxdepth 1 -name "sub-*"| sort -V)
IFS=$'\n' read -d '' -r -a subjects_array <<< "$subjects_list"
echo "subjects list are: $subjects_list"

path_derivatives=$(find . -type d -name derivatives)
shim_methods=("pi" "quadprog" "lsq" "grad")

for method in "${shim_methods[@]}"; do

    for subject in "${subjects_array[@]}"; do
        echo "Subject: $subject"
        echo "Method: $method"
        subject_name=$(basename "$subject")
        fmap_path=${path_derivatives}/${subject_name}
        mask=$(find ${fmap_path} -type f -name "mask_mrs.nii.gz")
        output_grad="${fmap_path}/fmap/static_shim_grad/MRS_mask"
        output_lsq="${fmap_path}/fmap/static_shim_lsq/MRS_mask"
        output_pi="${fmap_path}/fmap/static_shim_pi/MRS_mask"
        output_quadprog="${fmap_path}/fmap/static_shim_quadprog/MRS_mask"

        fieldmap=$(find ${path_derivatives}/${subject_name}/fmap -type f -name "${subject_name}_acq-${method}-before-Shim-fieldmap.nii.gz")
        echo $fieldmap


        all_subjects_path="${path_derivatives}/all_subjects"
        if [ ! -d "$all_subjects_path" ]; then
          mkdir "$all_subjects_path"
        fi
        # Create a file to store the time taken for each method
        time_file="${all_subjects_path}/${method}_MRS_mask_time.txt"

        if [ "${method}" = "lsq" ]; then
            { time -p st_b0shim dynamic --fmap "${fieldmap}" --anat "${fieldmap}" --mask "${mask}" --mask-dilation-kernel-size "5"  --optimizer-method "least_squares" --slices "volume" --output-file-format-scanner "chronological-coil" --scanner-coil-order "0,1,2" --output-value-format "absolute" --output "${output_lsq}"; } 2>&1 | grep real | cut -d' ' -f2 >> "${time_file}" || exit

        elif [ "${method}" = "grad" ]; then
            { time -p st_b0shim dynamic --fmap "${fieldmap}" --anat "${fieldmap}" --mask "${mask}" --mask-dilation-kernel-size "5"  --optimizer-method "gradient" --optimizer-criteria "ps_huber" --slices "volume" --output-file-format-scanner "chronological-coil" --scanner-coil-order "0,1,2" --output-value-format "absolute" --output "${output_grad}"; } 2>&1 | grep real | cut -d' ' -f2 >> "${time_file}" || exit

        elif [ "${method}" = "pi" ]; then
            { time -p st_b0shim dynamic --fmap "${fieldmap}" --anat "${fieldmap}" --mask "${mask}" --mask-dilation-kernel-size "5" --optimizer-method "pseudo_inverse" --slices "volume" --output-file-format-scanner "chronological-coil" --scanner-coil-order "0,1,2" --output-value-format "absolute" --output "${output_pi}"; } 2>&1 | grep real | cut -d' ' -f2 >> "${time_file}" || exit

        elif [ "${method}" = "quadprog" ]; then
            { time -p st_b0shim dynamic --fmap "${fieldmap}" --anat "${fieldmap}" --mask "${mask}" --mask-dilation-kernel-size "5" --optimizer-method "quad_prog" --slices "volume" --output-file-format-scanner "chronological-coil" --scanner-coil-order "0,1,2" --output-value-format "absolute" --output "${output_quadprog}"; } 2>&1 | grep real | cut -d' ' -f2 >> "${time_file}" || exit

        fi
    done
done

In [ ]:
### Display the average runtime for each of the shimming methods

def calculate_average(file_path):
    with open(file_path, 'r') as file:
        times = [float(line.strip()) for line in file]
        return sum(times) / len(times)

path = current_path+"/derivatives/all_subjects"
shim_methods = ["pi", "quadprog", "lsq", "grad"]

for method in shim_methods:
    file_path = os.path.join(path, f"{method}_MRS_mask_time.txt")
    if os.path.exists(file_path):
        average_time = np.round(calculate_average(file_path), 1)
        print(f"Average computation time for {method} in MRS mask: {average_time} seconds")
    else:
        print(f"File not found for {method}")

## 6. B0 Fieldmaps comparison

### 6.1 Function to analyze the fieldmaps

In [34]:
def analyse_fieldmap(source_dir, labels, fieldmaps, output, mask, subject, roi_name, fig_size, colors, y_offset):
    """
    Description: Generates a violin plot representing the distribution of the given fieldmaps
    labels: List of strings including fieldmaps label
    fieldmaps: List of numpy arrays of fieldmaps in the same order as labels
    output: Output folder to save the violin plot
    mask: Numpy array of a Mask corresponding to the MRS voxel volume
    """
    plt.figure(figsize=fig_size)
    sns.set(font_scale=2)

    output_path = os.path.join(source_dir, output)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    if np.shape(mask)==np.shape(fieldmaps[0]):
        dict_map_avg = {label: fieldmap[mask == 1] for label, fieldmap in zip(labels, fieldmaps)}
    else:
        dict_map_avg = {label: fieldmap for label, fieldmap in zip(labels, fieldmaps)}

    def calculate_rmse(data):
        mse = np.mean(np.square(data))
        rmse = np.sqrt(mse)
        return rmse

    dict_means = {key: np.round(np.mean(value), 1) for key, value in dict_map_avg.items()}
    dict_stds = {key: np.round(np.std(value), 1) for key, value in dict_map_avg.items()}
    dict_rmse = {key: calculate_rmse(value) for key, value in dict_map_avg.items()}

    ax = sns.violinplot(data=list(dict_map_avg.values()), palette=colors)

    # Get the limits of the y-axis to position the text
    ymin, ymax = ax.get_ylim()
    for i, (key, mean) in enumerate(dict_means.items()):
        mean_str = "{:.1f}".format(mean)
        std_str = "{:.1f}".format(dict_stds[key])
        rmse_str = "{:.1f}".format(dict_rmse[key])
        ax.text(i + 0.05, ymin + y_offset * (ymax - ymin), f"SD: {std_str}",
                horizontalalignment='left', verticalalignment='top', fontsize=14)

    ax.set_ylabel('Frequency [Hz]', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, fontsize=14 , fontweight='bold', rotation=25)
    ax.set_title(f"{subject}: $\Delta$B0 distribution_within_{roi_name}", fontsize=18, fontweight='bold')

    ax.tick_params(axis='y', labelsize=14)
    plt.savefig(os.path.join(output_path, f"{subject}_B0_distribution_within_{roi_name}_mask.png"), dpi=300)
    plt.show()

    return dict_stds


### 6.2 Display the fieldmaps distribution

In [ ]:
all_Grad_fmaps_mrs_mask_unshimmed=[]
all_PI_fmaps_mrs_mask_unshimmed=[]
all_LSq_fmaps_mrs_mask_unshimmed=[]
all_QuadProg_fmaps_mrs_mask_unshimmed=[]

all_Siemens_fmaps_mrs_mask_shimmed=[]
all_Grad_fmaps_mrs_mask_shimmed=[]
all_PI_fmaps_mrs_mask_shimmed=[]
all_LSq_fmaps_mrs_mask_shimmed=[]
all_QuadProg_fmaps_mrs_mask_shimmed=[]
sd_field_maps_mrs_mask = {}
methods = ['pi',  'quadprog','lsq', 'grad']

# Iterate over each subject
for subject in subjects_list:
    sd_field_maps_mrs_mask[subject] = {}
    derivative_path = current_path + f"/derivatives/{subject}/"
    mask_mrs_nii = nib.load(derivative_path + 'mask_mrs.nii.gz')
    mask_mrs = mask_mrs_nii.get_fdata()

    unshimmed_fieldmap_grad = nib.load (derivative_path + f"fmap/{subject}_acq-grad-before-Shim-fieldmap.nii.gz").get_fdata()
    unshimmed_fieldmap_pi = nib.load (derivative_path + f"fmap/{subject}_acq-pi-before-Shim-fieldmap.nii.gz").get_fdata()
    unshimmed_fieldmap_lsq= nib.load (derivative_path + f"fmap/{subject}_acq-lsq-before-Shim-fieldmap.nii.gz").get_fdata()
    unshimmed_fieldmap_quadprog = nib.load (derivative_path + f"fmap/{subject}_acq-quadprog-before-Shim-fieldmap.nii.gz").get_fdata()

    shimmed_fieldmap_siemens = nib.load (derivative_path + f"fmap/{subject}_acq-siemens-after-Shim-fieldmap.nii.gz").get_fdata()
    shimmed_fieldmap_grad = nib.load (derivative_path + f"fmap/{subject}_acq-grad-after-Shim-fieldmap.nii.gz").get_fdata()
    shimmed_fieldmap_pi = nib.load (derivative_path + f"fmap/{subject}_acq-pi-after-Shim-fieldmap.nii.gz").get_fdata()
    shimmed_fieldmap_lsq = nib.load (derivative_path + f"fmap/{subject}_acq-lsq-after-Shim-fieldmap.nii.gz").get_fdata()
    shimmed_fieldmap_quadprog  = nib.load (derivative_path + f"fmap/{subject}_acq-quadprog-after-Shim-fieldmap.nii.gz").get_fdata()

    for method in methods:
        fieldmap_unshimmed = f"unshimmed_fieldmap_{method}"
        fieldmap_shimmed = f"shimmed_fieldmap_{method}"
        fieldmap1 = eval(fieldmap_unshimmed)
        fieldmap2 = eval(fieldmap_shimmed)
        mean1 = np.mean(fieldmap1[mask_mrs > 0])
        mean2 = np.mean(fieldmap2[mask_mrs > 0])
        if mean1 > 0:
            fieldmap1[mask_mrs > 0] -= mean1
        else:
            fieldmap1[mask_mrs > 0] += abs(mean1)
        if mean2 > 0:
            fieldmap2[mask_mrs > 0] -= mean2
        else:
            fieldmap2[mask_mrs > 0] += abs(mean2)

    all_Grad_fmaps_mrs_mask_unshimmed.append(unshimmed_fieldmap_grad[mask_mrs==1].flatten())
    all_PI_fmaps_mrs_mask_unshimmed.append(unshimmed_fieldmap_pi[mask_mrs==1].flatten())
    all_LSq_fmaps_mrs_mask_unshimmed.append(unshimmed_fieldmap_lsq[mask_mrs==1].flatten())
    all_QuadProg_fmaps_mrs_mask_unshimmed.append(unshimmed_fieldmap_quadprog[mask_mrs==1].flatten())

    all_Siemens_fmaps_mrs_mask_shimmed.append(shimmed_fieldmap_siemens[mask_mrs==1].flatten())
    all_Grad_fmaps_mrs_mask_shimmed.append(shimmed_fieldmap_grad[mask_mrs==1].flatten())
    all_PI_fmaps_mrs_mask_shimmed.append(shimmed_fieldmap_pi[mask_mrs==1].flatten())
    all_LSq_fmaps_mrs_mask_shimmed.append(shimmed_fieldmap_lsq[mask_mrs==1].flatten())
    all_QuadProg_fmaps_mrs_mask_shimmed.append(shimmed_fieldmap_quadprog[mask_mrs==1].flatten())

    fmaps = [unshimmed_fieldmap_pi, shimmed_fieldmap_pi,
             unshimmed_fieldmap_quadprog, shimmed_fieldmap_quadprog,
             unshimmed_fieldmap_lsq, shimmed_fieldmap_lsq,
             unshimmed_fieldmap_grad, shimmed_fieldmap_grad,
             shimmed_fieldmap_siemens]

    labels = ["Unshimmed PI", "Shimmed PI",
              "Unshimmed QuadProg", "Shimmed QuadProg",
              "Unshimmed LSq", "Shimmed LSq",
              "Unshimmed Grad", "Shimmed Grad",
              "Shimmed Siemens"]

    colors = ['orange', 'orange', 'teal', 'teal',
              'pink', 'pink', 'cyan', 'cyan', 'gold']
    output = derivative_path
    sd = analyse_fieldmap(current_path, labels, fmaps, output, mask_mrs, subject, roi_name='MRS voxel',fig_size=(16,12), colors=colors, y_offset=0.3)
    sd_field_maps_mrs_mask[subject] =sd


In [ ]:
# Calculate standard deviation for each method across all subjects
std_dict = {}
for key in sd_field_maps_mrs_mask['sub-2'].keys():
    values = [sd_field_maps_mrs_mask[sub][key] for sub in sd_field_maps_mrs_mask]
    std_dict[key] = np.round(np.std(values),1)

print("Standard Deviation for Each method across all subjects:")
for key, value in std_dict.items():
    print(f"{key}: {value} Hz")

In [ ]:
### Display the fieldmaps distributions for all subjects
shim_methods=[ "PI", "QuadProg", "LSq", "Grad"]

for method in shim_methods:
    # Create the variable name dynamically
    var_name_unshimmed = f"all_{method}_data_mrs_mask_unshimmed"
    var_name_shimmed = f"all_{method}_data_mrs_mask_shimmed"
    var_Siemens= f"all_Siemens_data_mrs_mask_shimmed"
    # Extract the data for the current method
    data_unshimmed = locals()[f"all_{method}_fmaps_mrs_mask_unshimmed"]
    data_shimmed = locals()[f"all_{method}_fmaps_mrs_mask_shimmed"]
    data_Siemens = locals()[f"all_Siemens_fmaps_mrs_mask_shimmed"]

    concat_data_unshimmed = np.concatenate(data_unshimmed)
    concat_data_shimmed = np.concatenate(data_shimmed)
    concat_data_Siemens = np.concatenate(data_Siemens)

    # Assign the concat_data to the dynamically created variable
    exec(f"{var_name_unshimmed} = concat_data_unshimmed")
    exec(f"{var_name_shimmed} = concat_data_shimmed")
    exec(f"{var_Siemens} = concat_data_Siemens")

fmaps = [all_PI_data_mrs_mask_unshimmed, all_PI_data_mrs_mask_shimmed, all_QuadProg_data_mrs_mask_unshimmed, all_QuadProg_data_mrs_mask_shimmed,
         all_LSq_data_mrs_mask_unshimmed, all_LSq_data_mrs_mask_shimmed, all_Grad_data_mrs_mask_unshimmed, all_Grad_data_mrs_mask_shimmed, all_Siemens_data_mrs_mask_shimmed]
labels = ["Unshimmed PI", "Shimmed PI", "Unshimmed QuadProg", "Shimmed QuadProg", "Unshimmed LSq", "Shimmed LSq", "Unshimmed Grad", "Shimmed Grad", "Shimmed Siemens"]

source_dir = os.getcwd()
output = 'derivatives/all_subjects'
subject = 'all subjects'
colors = ['orange', 'orange',  'teal', 'teal',
          'pink', 'pink',  'cyan', 'cyan', 'gold']
distribution = analyse_fieldmap(source_dir, labels, fmaps, output, mask_mrs, subject, roi_name='MRS',fig_size=(16,14), colors=colors, y_offset=0.3)


### 6.3 Perform statistical test on fieldmaps

By comparing the variance, the following cell assesses whether the distribution of each shimming method significantly differs from the Siemens shimming.
The documentation for the following tests can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html) and [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html).


In [ ]:
import statsmodels.stats.multitest as smm
# A Kolmogorov-Smirnov test to check the normality of the reference distribution (Siemens shimming)
statistic, p_value_normality = stats.kstest(all_Siemens_data_mrs_mask_shimmed, 'norm')
# Set significant level
alpha = 0.01

# Interpret the results
# H0: the sample has a Gaussian distribution.
# H1: the sample does not have a Gaussian distribution.
if p_value_normality < alpha:
    print('Reject the null hypothesis. The provided data does not follow a normal distribution.')
else:
    print("Fail to reject the null hypothesis. The provided data follow a normal distribution.")

comparison_methods = ['PI','Grad','QuadProg','LSq']
Uncor_pvals = []

# levene test
for method in comparison_methods:
    data = eval(f"all_{method}_data_mrs_mask_shimmed")
    if np.var(data)< np.var(all_Siemens_data_mrs_mask_shimmed):
      statistic, p_value = stats.levene(all_Siemens_data_mrs_mask_shimmed, data)
      Uncor_pvals.append(p_value)
    else:
      comparison_methods.remove(method)
      print(f"The variance of the {method} is higher than Siemens and excluded from further comparisons.")

print("Uncorrected p-values are: ", Uncor_pvals)
# Perform Multiple comparison correction with step-down method using Bonferroni adjustments
Corrected_pvals = smm.multipletests(Uncor_pvals, alpha=0.01, method='holm', is_sorted=False, returnsorted=False)
print("Corrected p-values are: " , Corrected_pvals[1])

for pval, method in zip(Corrected_pvals[1], comparison_methods):
  if pval < 0.01:
    print(f"Reject the null hypothesis. The variance between the Siemens and {method} methods are significantly different.")
  else:
    print(f"Fail to reject the null hypothesis. There is no significant difference in variances between the Siemens and {method} methods.")


In [ ]:
# Comparing Grad method to other methods

comparison_methods= ['PI', 'QuadProg', 'LSq']
# levene test
for method in comparison_methods:
    data = eval(f"all_{method}_data_mrs_mask_shimmed")
    statistic, p_value = stats.levene(all_Grad_data_mrs_mask_shimmed, data)

    # Set significancy level
    alpha = 0.01

    # Interpret the results
    if p_value < alpha:
        print(f"Reject the null hypothesis. The variance between the Grad and {method} groups are significantly different.")
    else:
        print(f"Fail to reject the null hypothesis. There is no significant difference in variances between the Grad and {method} groups.")


In [ ]:
# Comparing PI method to other methods

comparison_methods= ['QuadProg', 'LSq']
# levene test
for method in comparison_methods:
    data = eval(f"all_{method}_data_mrs_mask_shimmed")
    statistic, p_value = stats.levene(all_PI_data_mrs_mask_shimmed, data)

    # Set significancy level
    alpha = 0.01

    # Interpret the results
    if p_value < alpha:
        print(f"Reject the null hypothesis. The variance between the PI and {method} groups are significantly different.")
    else:
        print(f"Fail to reject the null hypothesis. There is no significant difference in variances between the PI and {method} groups.")


## 7. MRS analysis

###  7.1. Water removal and fitting

In [ ]:
%%bash

subjects_list=$(find .  -type d -maxdepth 1 -name "sub-*")
IFS=$'\n' read -d '' -r -a subjects_array <<< "$subjects_list"
echo "subjects list are: $subjects_list"

path_derivatives=$(find . -type d -name derivatives)

for subject in "${subjects_array[@]}"; do
    echo "Analyzing subject: ${subject}"
    path_mrs=$(find ${subject} -type d -name mrs)
    echo "mrs path is: ${path_mrs}"
    path_t1=$(find ${subject} -type f -name *T1w.nii.gz)

    sub_basename=$(basename "${subject}")
    subject_derivatives=${path_derivatives}/${sub_basename}/mrs
    echo "${sub_basename} derivatives path is: ${subject_derivatives}"

    shim_methods=("siemens" "pi" "quadprog" "lsq" "grad")
    for method in "${shim_methods[@]}"; do
        fsl_mrs_proc remove --file ${path_mrs}/${subject}_acq-press-${method}-shim_nuc-H_echo-135_svs.nii.gz --output ${subject_derivatives}/${method}_water_removed -r  --ppm 3.5 9
        fsl_mrs  --data ${subject_derivatives}/${method}_water_removed/${subject}_acq-press-${method}-shim_nuc-H_echo-135_svs.nii.gz --basis ${path_derivatives}/LCModel_Siemens_UnEdited_PRESS_135_ALL.BASIS --t1 ${path_t1} --output ${subject_derivatives}/${method}_fit_result --baseline_order 2   --internal_ref 'Cr' --ignore H2O PCr GPC  --report --overwrite --ppmlim 0 4 --TE 135 --TR 2
    done
done


### 7.2 Plot the fittings

In [ ]:
current_path = os.getcwd()
index = 1

# Create a figure with a 5x5 grid of subplots
fig, axes = plt.subplots(5, 5, figsize=(40, 40))

# Reduce the gap between subplots
plt.subplots_adjust(wspace=0, hspace=0.01)

shim_methods = ['siemens', 'pi', 'quadprog', 'lsq', 'grad']

for i, subject in enumerate(subjects_list):
    for j, method in enumerate(shim_methods):
        ax = axes[i, j]
        fit_path = os.path.join(current_path, "derivatives", subject, "mrs", f"{method}_fit_result")
        ax.imshow(plt.imread(os.path.join(fit_path, 'fit_summary.png')))
        ax.set_xticks([])
        ax.set_yticks([])

        # Add titles for the first row
        if i == 0:
            ax.set_title(method.capitalize(), fontsize=32,fontweight='bold', pad=20)

        # Add labels for the first column
        if j == 0:
            ax.set_ylabel(subject, fontsize=32, fontweight='bold',labelpad=20)

# Save the plot with tight layout
plt.savefig(os.path.join(current_path, 'derivatives', 'all_subjects', 'mrs_fittings.png'), dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


### 7.3. Extract the metabolites fit results from the CSV files

In [ ]:
# path to the current directory
current_path = os.getcwd()

# Iterate over each subject and shim method to extract fit results
crlb_NAA_all = {}
fwhm_NAA_all = {}
snr_NAA_all = {}

index = 1
for subject in subjects_list:
    crlb_NAA_all[subject] = {}
    fwhm_NAA_all[subject] = {}
    snr_NAA_all[subject] = {}

    for method in shim_methods:
        fit_path = os.path.join(current_path, f"derivatives/{subject}/mrs/{method}_fit_result/")

        # Read summary.csv file
        summary_file_path = os.path.join(fit_path, 'summary.csv')
        if os.path.exists(summary_file_path):
            df = pd.read_csv(summary_file_path)

            # Get the fit results for NAA
            CRLB_NAA = df.at[11, '%CRLB']
            fwhm_NAA = df.at[11, 'FWHM']
            snr_NAA = df.at[11, 'SNR']
            crlb_NAA_all[subject][method] = CRLB_NAA
            fwhm_NAA_all[subject][method] = fwhm_NAA
            snr_NAA_all[subject][method] = snr_NAA

### 7.4. Comparing NAA's FWHM, SNR and CRLB across differnt shim methods

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.markers as mmarkers

# Create figure and axes
fig, axs = plt.subplots(1, 3, figsize=(18, 6))


# Compute the mean values for FWHM, SNR, and CRLB for NAA
mean_fwhm = {key: np.round(np.mean([fwhm_NAA_all[sub][key] for sub in fwhm_NAA_all]), decimals=1) for key in fwhm_NAA_all['sub-1'].keys()}
mean_snr = {key: np.round(np.mean([snr_NAA_all[sub][key] for sub in snr_NAA_all]), decimals=1) for key in snr_NAA_all['sub-1'].keys()}
mean_crlb = {key: np.round(np.mean([crlb_NAA_all[sub][key] for sub in crlb_NAA_all]), decimals=1) for key in crlb_NAA_all['sub-1'].keys()}

# Define a horizontal line marker
hline_marker = mmarkers.MarkerStyle('_')
hline_marker._transform = hline_marker.get_transform().scale(5, 5)

# Plot FWHM
axs[0].set_title("FWHM (Hz) NAA", fontsize=18)
for sub in fwhm_NAA_all:
    axs[0].scatter(list(fwhm_NAA_all[sub].keys()), list(fwhm_NAA_all[sub].values()), label=sub)
axs[0].scatter(list(mean_fwhm.keys()), list(mean_fwhm.values()), color='black', marker=hline_marker, label='mean')
axs[0].legend(prop={'size': 15}, loc='upper right', bbox_to_anchor=(0.4, 1))
axs[0].tick_params(axis='x', rotation=45, labelsize=14)
axs[0].yaxis.set_minor_locator(plt.MultipleLocator(1))  # Set minor ticks for y-axis
axs[0].grid(which='major', axis='x')

# Plot SNR
axs[1].set_title("SNR (A.U.) NAA", fontsize=18)
for sub in snr_NAA_all:
    axs[1].scatter(list(snr_NAA_all[sub].keys()), list(snr_NAA_all[sub].values()), label=sub)
axs[1].scatter(list(mean_snr.keys()), list(mean_snr.values()), color='black', marker=hline_marker, label='mean')
axs[1].legend(prop={'size': 15}, loc='upper right', bbox_to_anchor=(0.9, 1))
axs[1].tick_params(axis='x', rotation=45, labelsize=14)
axs[1].yaxis.set_minor_locator(plt.MultipleLocator(1))  # Set minor ticks for y-axis
axs[1].grid(which='major', axis='x')

# Plot CRLB
axs[2].set_title("CRLB (%) NAA", fontsize=18)
for sub in crlb_NAA_all:
    axs[2].scatter(list(crlb_NAA_all[sub].keys()), list(crlb_NAA_all[sub].values()), label=sub)
axs[2].scatter(list(mean_crlb.keys()), list(mean_crlb.values()), color='black', marker=hline_marker, label='mean')
axs[2].legend(prop={'size': 15}, loc='upper right', bbox_to_anchor=(0.5, 1))
axs[2].tick_params(axis='x', rotation=45, labelsize=14)
axs[2].yaxis.set_minor_locator(plt.MultipleLocator(1))  # Set minor ticks for y-axis
axs[2].grid(which='major', axis='x')

# Set x tick labels
axs[0].set_xticklabels(["Siemens", "PI", "QuadProg", "LSq", "Grad"])
axs[1].set_xticklabels(["Siemens", "PI", "QuadProg", "LSq", "Grad"])
axs[2].set_xticklabels(["Siemens", "PI", "QuadProg", "LSq", "Grad"])

# Adjust layout
fig.tight_layout()
plt.savefig(os.path.join(current_path, 'derivatives', 'all_subjects', 'FWHM_SNR_comparison.png'), dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


### 7.5. Create a CSV output  

In [ ]:

# Initialize lists for each criterion
all_data_dicts_fwhm = []
all_data_dicts_snr = []
all_data_dicts_crlb = []
current_path = os.getcwd()
criteria_list = ['fwhm', 'snr', 'crlb']

# Create a dictionary for easy access to mean values
mean_criteria_dict = {
    'fwhm': mean_fwhm,
    'snr': mean_snr,
    'crlb': mean_crlb
}

for criteria in criteria_list:
    mean_criteria = mean_criteria_dict[criteria]

    # Calculate the percentage difference for each dictionary
    percentage_diff_dict = {}
    siemens_value = mean_criteria['siemens']
    percentage_diff = {key: ((value - siemens_value) / siemens_value) * 100 for key, value in mean_criteria.items() if key != 'siemens'}

    combined_dict = {'criteria': criteria.upper()}
    for key, value in mean_criteria.items():
        combined_dict[f'{key}'] = round(value, 1)  # Round to one decimal place
        if key != 'siemens':
            combined_dict[f'{key} change (%)'] = round(percentage_diff.get(key, None), 1)  # Round to one decimal place

    if criteria == 'fwhm':
        all_data_dicts_fwhm.append(combined_dict)
    elif criteria == 'snr':
        all_data_dicts_snr.append(combined_dict)
    elif criteria == 'crlb':
        all_data_dicts_crlb.append(combined_dict)

# Create DataFrames for each criterion
df_fwhm = pd.DataFrame(all_data_dicts_fwhm)
df_snr = pd.DataFrame(all_data_dicts_snr)
df_crlb = pd.DataFrame(all_data_dicts_crlb)

# Display the DataFrames
df_fwhm_formatted = df_fwhm.apply(lambda x: x.apply(lambda y: '{:.1f}'.format(y) if isinstance(y, float) else y))
df_snr_formatted = df_snr.apply(lambda x: x.apply(lambda y: '{:.1f}'.format(y) if isinstance(y, float) else y))
df_crlb_formatted = df_crlb.apply(lambda x: x.apply(lambda y: '{:.1f}'.format(y) if isinstance(y, float) else y))

# Display the formatted DataFrames
print("FWHM DataFrame:")
display(df_fwhm_formatted)

print("SNR DataFrame:")
display(df_snr_formatted)

print("CRLB DataFrame:")
display(df_crlb_formatted)

# Create separator rows
separator_row_snr = pd.DataFrame({'criteria': ['SNR']})
separator_row_crlb = pd.DataFrame({'criteria': ['CRLB']})
separator_row_columns = pd.DataFrame([df_fwhm_formatted.columns], columns=df_fwhm_formatted.columns)

# Concatenate the DataFrames with the separator rows
combined_df = pd.concat([df_fwhm_formatted, separator_row_snr, separator_row_columns, df_snr_formatted, separator_row_crlb, separator_row_columns, df_crlb_formatted], axis=0)

# Save the combined DataFrame to a CSV file
combined_df.to_csv(current_path + '/derivatives/all_subjects/Metabolite_fit.csv', index=False)